In [2]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from transformers import AdamW,T5ForConditionalGeneration,T5Tokenizer,get_linear_schedule_with_warmup
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    self.hparams = hparams
    
    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)
  
  def is_logger(self):
    return self.trainer.proc_rank <= 0
  
  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        lm_labels=lm_labels,
    )

  def _step(self, batch):
    lm_labels = batch["target_ids"]
    lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        lm_labels=lm_labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]

    return loss

  def training_step(self, batch, batch_idx):
    loss = self._step(batch)

    tensorboard_logs = {"train_loss": loss}
    return {"loss": loss, "log": tensorboard_logs}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    tensorboard_logs = {"avg_train_loss": avg_train_loss}
    return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def validation_step(self, batch, batch_idx):
    loss = self._step(batch)
    return {"val_loss": loss}
  
  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    tensorboard_logs = {"val_loss": avg_loss}
    return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None):
    if self.trainer.use_tpu:
      xm.optimizer_step(optimizer)
    else:
      optimizer.step()
    optimizer.zero_grad()
    self.lr_scheduler.step()
  
  def get_tqdm_dict(self):
    tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

    return tqdm_dict

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
    dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
    return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

[nltk_data] Downloading package punkt to /home/amadeus/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
INFO:transformers.file_utils:PyTorch version 1.11.0+cu102 available.


In [3]:
import pandas as pd

train_df = pd.read_csv('./content/train.csv') # 1179
test_df = pd.read_csv('/data/test_phase_2_update.csv') # 150

In [3]:
train_examples = []
for index, row in train_df.iterrows():
  # print (row)
  claim, evidence, label = row['Claim'], row['Evidence'], row['Label']
  train_examples.append([claim.strip(), evidence.strip(), label])

random.shuffle(train_examples)
valid_num = int(len(train_examples)*0.1)
valid_examples = train_examples[:valid_num]
train_examples = train_examples[valid_num:]

test_examples = []
for index, row in test_df.iterrows():
  # print (row)
  claim, evidence = row['Claim'], row['Evidence']
  test_examples.append([claim.strip(), evidence.strip()])

In [4]:
label_int = {"NonInfo":0,"Supports":1, 'Refutes':2}
labels = ['NonInfo',"Supports","Refutes"]

In [5]:
class FactDataset(Dataset):
  def __init__(self, tokenizer,data,  max_len=512):
    
    self.data = data
    self.max_len = max_len
    self.tokenizer = tokenizer
    self.inputs = []
    self.targets = []

    self._build()
  
  def __len__(self):
    return len(self.inputs)
  
  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
    target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

    return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}
  
  def _build(self):
    for simple in self.data:
      if len(simple) == 3:
        input_, target = "Claim:"+ simple[0] + "Evidence:" + simple[1], labels[simple[2]]    
      else:
        input_, target = "Claim:"+ simple[0] + "Evidence:" + simple[1], labels[0]    

      input_ = input_ + ' </s>'
      target = target + " </s>"

       # tokenize inputs
      tokenized_inputs = self.tokenizer.batch_encode_plus(
          [input_], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt"
      )
       # tokenize targets
      tokenized_targets = self.tokenizer.batch_encode_plus(
          [target], max_length=2, pad_to_max_length=True, return_tensors="pt"
      )

      self.inputs.append(tokenized_inputs)
      self.targets.append(tokenized_targets)

In [8]:
train_dataset =FactDataset(tokenizer,train_examples, 512)
print(len(train_dataset))
valid_dataset = FactDataset(tokenizer,valid_examples, 512)
print(len(valid_dataset))

1062
117


In [7]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
for label in labels:
  print(len(tokenizer.encode(label)))

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/t5-spiece.model from cache at /home/amadeus/.cache/torch/transformers/68f1b8dbca4350743bb54b8c4169fd38cbabaad564f85a9239337a8d0342af9f.9995af32582a1a7062cb3173c118cb7b4636fa03feb967340f20fc37406f021f


2
2
4


In [4]:
args_dict = dict(
    data_dir="", # path for data files
    output_dir="", # path to save the checkpoints
    model_name_or_path='t5-base',
    tokenizer_name_or_path='t5-base',
    max_seq_length=512,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=2,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)
args_dict.update({'data_dir': 'fact_data', 'output_dir': '/data/t5_fact', 'num_train_epochs':20})
args = argparse.Namespace(**args_dict)
print(args_dict)

{'data_dir': 'fact_data', 'output_dir': '/data/t5_fact', 'model_name_or_path': 't5-base', 'tokenizer_name_or_path': 't5-base', 'max_seq_length': 512, 'learning_rate': 0.0003, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'train_batch_size': 8, 'eval_batch_size': 8, 'num_train_epochs': 20, 'gradient_accumulation_steps': 16, 'n_gpu': 1, 'early_stop_callback': False, 'fp_16': False, 'opt_level': 'O1', 'max_grad_norm': 1.0, 'seed': 42}


In [5]:
model = T5FineTuner(args)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json from cache at /home/amadeus/.cache/torch/transformers/40578967d1f029acb6162b36db9d8b4307063e885990ccd297c2c5be1cf1b3d7.2995d650f5eba18c8baa4146e210d32d56165e90d374281741fc78b872cd6c9b
INFO:transformers.configuration_utils:Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size"

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://cdn.huggingface.co/t5-base-pytorch_model.bin in cache at /home/amadeus/.cache/torch/transformers/f6f2fde9fa7611f4eff74620de9cbe734e7a717b5b143bd283cae4c2d6022990.54f906ff53bd09195cfc183a29cadc81b7705f07fcdb796d24163cb632b6bdfa
INFO:transformers.file_utils:creating metadata file for /home/amadeus/.cache/torch/transformers/f6f2fde9fa7611f4eff74620de9cbe734e7a717b5b143bd283cae4c2d6022990.54f906ff53bd09195cfc183a29cadc81b7705f07fcdb796d24163cb632b6bdfa
INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/t5-base-pytorch_model.bin from cache at /home/amadeus/.cache/torch/transformers/f6f2fde9fa7611f4eff74620de9cbe734e7a717b5b143bd283cae4c2d6022990.54f906ff53bd09195cfc183a29cadc81b7705f07fcdb796d24163cb632b6bdfa
INFO:transformers.modeling_utils:Weights of T5ForConditionalGeneration not initialized from pretrained model: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
INFO:transformers.to

state_dict in:https://drive.google.com/file/d/1Cu4M8ca22jt_7aLwUkow5rt-XRGEgbT9/view?usp=sharing

In [6]:
state = torch.load("/data/t5_fact/checkpointepoch=7_v0.ckpt")
model.load_state_dict(state['state_dict'])
model = model.cuda()

/home/amadeus/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [27]:
dataset = FactDataset(tokenizer, test_examples, 512)
loader = DataLoader(dataset, batch_size=32, num_workers=4)
model.model.eval()
outputs = []

for batch in tqdm(loader):
  outs = model.model.generate(input_ids=batch['source_ids'].cuda(), 
                              attention_mask=batch['source_mask'].cuda(), 
                              max_length=2)

  dec = [tokenizer.decode(ids) for ids in outs]
  # print(len(dec))
  
  # print(len(target))
  outputs.extend(dec)
  

  0%|          | 0/5 [00:00<?, ?it/s]

In [28]:
results= []
for i in outputs:
  if i == "Re":
    results.append(2)
  elif i == "Non":
    results.append(0)
  elif i == "Support":
    results.append(1)
  else:
    print("error")

This is the result of original task

In [29]:

for i in results:
    print(i)

0
0
1
1
1
0
0
1
2
1
1
1
1
2
1
1
0
2
0
0
2
1
2
1
1
2
1
1
1
0
1
2
2
1
1
1
1
1
1
0
0
1
0
2
1
1
1
2
0
1
1
1
0
1
1
0
2
1
1
1
0
1
0
2
2
1
2
0
0
1
1
0
0
1
1
1
1
1
0
0
1
0
1
0
1
2
1
1
1
1
2
0
0
1
0
0
2
2
2
2
2
0
1
1
0
0
1
1
1
1
0
1
2
0
2
2
2
1
0
2
2
1
1
1
1
1
1
2
1
0
1
2
2
1
0
2
1
1
0
1
2
0
1
0
1
0
2
0
0
1


state_dict in https://drive.google.com/file/d/1Wtos9PjCyASryzokSezfmaSMTbexLwYF/view?usp=sharing

In [46]:
state = torch.load("/data/t5_info/checkpointepoch=9.ckpt")
model.load_state_dict(state['state_dict'])
model = model.cuda()

In [47]:
dataset = FactDataset(tokenizer, test_examples, 512)
loader = DataLoader(dataset, batch_size=32, num_workers=4)
model.model.eval()
outputs = []

for batch in tqdm(loader):
  outs = model.model.generate(input_ids=batch['source_ids'].cuda(), 
                              attention_mask=batch['source_mask'].cuda(), 
                              max_length=2)

  dec = [tokenizer.decode(ids) for ids in outs]
  # print(len(dec))
  
  # print(len(target))
  outputs.extend(dec)
  

  0%|          | 0/5 [00:00<?, ?it/s]

In [48]:
print(outputs)

['Non', 'Non', 'Info', 'Info', 'Info', 'Non', 'Non', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Non', 'Info', 'Non', 'Non', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Non', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Non', 'Non', 'Info', 'Non', 'Non', 'Info', 'Info', 'Info', 'Info', 'Non', 'Info', 'Info', 'Info', 'Non', 'Info', 'Info', 'Non', 'Info', 'Info', 'Info', 'Info', 'Non', 'Info', 'Non', 'Info', 'Info', 'Info', 'Info', 'Non', 'Non', 'Info', 'Info', 'Non', 'Non', 'Info', 'Info', 'Info', 'Info', 'Info', 'Non', 'Non', 'Info', 'Non', 'Info', 'Non', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Non', 'Non', 'Info', 'Non', 'Non', 'Info', 'Info', 'Info', 'Info', 'Info', 'Non', 'Info', 'Info', 'Non', 'Non', 'Info', 'Info', 'Info', 'Info', 'Non', 'Info', 'Info', 'Non', 'Info', 'Info', 'Info', 'Info', 'Non', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'N

In [33]:
len(outputs)

150

In [49]:
results= []
for i in outputs:
  if i == "Non":
    results.append(0)
  elif i == "Info":
    results.append(1)
  else:
    print("error")

In [35]:
fact_results

[0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1]

state_dict in https://drive.google.com/file/d/1thnZqHIslJNid3ZJ-v4yfgfLSDGTm2Xl/view?usp=sharing

In [36]:
state = torch.load("/data/t5_fact_check/checkpointcheckpoint_ckpt_epoch_10.ckpt")
model.load_state_dict(state['state_dict'])
model = model.cuda()
dataset = FactDataset(tokenizer, test_examples, 512)
loader = DataLoader(dataset, batch_size=32, num_workers=4)
model.model.eval()
semantic_outputs = []

for batch in tqdm(loader):
  outs = model.model.generate(input_ids=batch['source_ids'].cuda(), 
                              attention_mask=batch['source_mask'].cuda(), 
                              max_length=2)

  dec = [tokenizer.decode(ids) for ids in outs]
  # print(len(dec))
  
  # print(len(target))
  semantic_outputs.extend(dec)
  

  0%|          | 0/5 [00:00<?, ?it/s]

In [37]:
semantic_outputs

['Re',
 'Support',
 'Support',
 'Support',
 'Support',
 'Support',
 'Re',
 'Re',
 'Re',
 'Re',
 'Support',
 'Re',
 'Support',
 'Re',
 'Support',
 'Support',
 'Re',
 'Re',
 'Support',
 'Support',
 'Re',
 'Support',
 'Re',
 'Support',
 'Support',
 'Re',
 'Support',
 'Support',
 'Re',
 'Re',
 'Support',
 'Re',
 'Re',
 'Re',
 'Support',
 'Support',
 'Support',
 'Re',
 'Re',
 'Support',
 'Re',
 'Support',
 'Support',
 'Re',
 'Support',
 'Support',
 'Support',
 'Re',
 'Support',
 'Support',
 'Support',
 'Support',
 'Support',
 'Support',
 'Support',
 'Support',
 'Re',
 'Support',
 'Support',
 'Support',
 'Support',
 'Re',
 'Support',
 'Re',
 'Re',
 'Support',
 'Re',
 'Support',
 'Re',
 'Support',
 'Support',
 'Support',
 'Support',
 'Support',
 'Support',
 'Support',
 'Support',
 'Re',
 'Support',
 'Re',
 'Support',
 'Re',
 'Re',
 'Support',
 'Support',
 'Re',
 'Support',
 'Support',
 'Re',
 'Support',
 'Re',
 'Support',
 'Support',
 'Support',
 'Support',
 'Support',
 'Re',
 'Re',
 'Re',
 '

In [39]:
semantic_results= []
for i in semantic_outputs:
  if i == "Re":
    semantic_results.append(2)
  elif i == "Support":
    semantic_results.append(1)
  else:
    print("error")

In [40]:
semantic_results

[2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1]

In [41]:
for i in range(len(results)):
    if results[i] == 0:
        continue
    else:
        results[i] = semantic_results[i]
    

In [42]:
results

[0,
 0,
 1,
 1,
 1,
 0,
 0,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 0,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 0,
 2,
 0,
 2,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 1,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 2,
 0,
 0,
 1]

In [43]:
len(results)

150

In [44]:
for i in results:
    print(i)

0
0
1
1
1
0
0
2
2
2
1
2
1
2
1
1
0
2
0
0
2
1
2
1
1
2
1
1
2
0
1
2
2
2
1
1
1
2
2
0
0
1
0
0
1
1
1
2
0
1
1
1
0
1
1
0
2
1
1
1
0
2
0
2
2
1
2
0
0
1
1
0
0
1
1
1
1
2
0
0
1
0
2
0
1
2
1
1
2
1
2
0
0
1
0
0
2
2
2
2
2
0
1
1
0
0
1
1
1
1
0
1
1
0
2
2
2
2
0
1
2
1
1
1
1
1
2
2
1
0
1
2
2
1
0
2
1
1
0
1
1
0
1
0
1
0
2
0
0
1


In [45]:
new_results =[0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 0,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 0,
 0,
 1,
 0,
 2,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 2,
 2,
 1,
 1,
 0,
 1,
 0,
 2,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 1,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 2,
 0,
 0,
 1]

In [50]:
for i in range(len(results)):
    if results[i] == 0:
        continue
    else:
        results[i] = new_results[i]
    

In [51]:
for i in results:
    print(i)

0
0
1
1
1
0
0
1
2
2
1
2
1
2
1
1
0
2
0
0
2
1
2
1
1
2
1
1
2
0
1
2
2
1
1
1
1
2
2
0
0
1
0
0
1
1
1
2
0
1
1
1
0
1
1
0
2
2
1
1
0
1
0
2
2
1
2
0
0
1
1
0
0
1
1
1
1
2
0
0
1
0
2
0
1
2
1
1
2
1
2
0
0
1
0
0
2
2
2
2
2
0
1
1
0
0
1
1
1
1
0
1
1
0
2
2
2
2
0
1
2
1
1
1
1
1
2
2
1
0
1
2
2
1
0
2
1
1
0
1
1
0
1
0
1
0
2
0
0
1


In [1]:
!nvidia-smi

/bin/sh: 1: nvidia-smi: not found
